### Retinal Disease Classification using different Deep Learning Models

Import required modules

In [1]:
import warnings
warnings.filterwarnings(action = 'ignore')

In [2]:
import os
import cv2

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-08-12 15:31:30.020254: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 15:31:30.020385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 15:31:30.175371: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading Data

In [3]:
# Path to Data
train_path = '/kaggle/input/Training_Set/Training_Set/Training'
test_path = '/kaggle/input/Test_Set/Test_Set/Test'
validation_path = '/kaggle/input/Evaluation_Set/Evaluation_Set/Validation'

train_labels_path = '/kaggle/input/Training_Set/Training_Set/RFMiD_Training_Labels.csv'
test_labels_path = '/kaggle/input/Test_Set/Test_Set/RFMiD_Testing_Labels.csv'
validation_labels_path = '/kaggle/input/Evaluation_Set/Evaluation_Set/RFMiD_Validation_Labels.csv'

In [4]:
!ls ../input/

Evaluation_Set	Test_Set  Training_Set


In [5]:
# Loading labels
train_labels = pd.read_csv(train_labels_path)
test_labels = pd.read_csv(test_labels_path)
val_labels = pd.read_csv(validation_labels_path)

In [6]:
print("Train shape:", train_labels.shape)
print("Test shape:", test_labels.shape)
print("Validation shape:", val_labels.shape)

Train shape: (1920, 47)
Test shape: (640, 47)
Validation shape: (640, 47)


In [7]:
# to dislpay all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [8]:
print(train_labels.head())

   ID  Disease_Risk  DR  ARMD  MH  DN  MYA  BRVO  TSLN  ERM  LS  MS  CSR  ODC  \
0   1             1   1     0   0   0    0     0     0    0   0   0    0    0   
1   2             1   1     0   0   0    0     0     0    0   0   0    0    0   
2   3             1   1     0   0   0    0     0     0    0   0   0    0    0   
3   4             1   0     0   1   0    0     0     0    0   0   0    0    1   
4   5             1   1     0   0   0    0     0     0    0   1   0    0    0   

   CRVO  TV  AH  ODP  ODE  ST  AION  PT  RT  RS  CRS  EDN  RPEC  MHL  RP  CWS  \
0     0   0   0    0    0   0     0   0   0   0    0    0     0    0   0    0   
1     0   0   0    0    0   0     0   0   0   0    0    0     0    0   0    0   
2     0   0   0    0    0   0     0   0   0   0    0    0     0    0   0    0   
3     0   0   0    0    0   0     0   0   0   0    0    0     0    0   0    0   
4     0   0   0    0    0   0     0   0   0   0    0    0     0    0   0    0   

   CB  ODPM  PRH  MNF  HR 

In [9]:
# Exploratory Data Analysis
disease_counts = train_labels.iloc[:, 2:].sum().sort_values(ascending=False)
#using plotly
fig = px.bar(disease_counts, title="Disease Distribution in Training Set") 
fig.show()

In [10]:
# Preprocessing the data
# function to load the images and labels
def load_images_and_labels(image_dir, labels_df):
    images = []
    labels = []
    for index, row in labels_df.iterrows():
        img_path = os.path.join(image_dir, f"{row['ID']}.png")
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            # Resizing images to 224x224
            img = cv2.resize(img, (224, 224))
            images.append(img)
            labels.append(row['Disease_Risk'])
    return np.array(images), np.array(labels)

# Load images and labels
x_train, y_train = load_images_and_labels(train_path, train_labels)
x_test, y_test = load_images_and_labels(test_path, test_labels)
x_val, y_val = load_images_and_labels(validation_path, val_labels)

In [11]:
# Normalizing the images
x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0

In [12]:
# Image Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

#### Defining the model - CNN

In [13]:
warnings.filterwarnings('ignore')
# Using Convolutional Neural Network

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Train the model

In [18]:
warnings.filterwarnings('ignore')

history = model.fit(
    datagen.flow(x_train, y_train, batch_size=32),
    validation_data=(x_val, y_val),
    epochs=20
)

Epoch 1/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 29s 349ms/step - accuracy: 0.7943 - loss: 0.4878 - val_accuracy: 0.7906 - val_loss: 0.4979
Epoch 2/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 23s 340ms/step - accuracy: 0.7875 - loss: 0.4885 - val_accuracy: 0.7906 - val_loss: 0.4773
Epoch 3/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 23s 336ms/step - accuracy: 0.7940 - loss: 0.4617 - val_accuracy: 0.7906 - val_loss: 0.4825
Epoch 4/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 23s 340ms/step - accuracy: 0.8002 - loss: 0.4578 - val_accuracy: 0.7906 - val_loss: 0.4440
Epoch 5/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 23s 339ms/step - accuracy: 0.7823 - loss: 0.4601 - val_accuracy: 0.7906 - val_loss: 0.4442
Epoch 6/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 23s 337ms/step - accuracy: 0.7798 - loss: 0.4784 - val_accuracy: 0.7906 - val_loss: 0.5632
Epoch 7/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 23s 332ms/step - accuracy: 0.7910 - loss: 0.4671 - val_accuracy: 0.7906 - val_loss: 0.4873
Epoch 8/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 23s 338ms/step - accuracy: 0.7951 - loss: 0.4489 - val_accu

Evaluate the model

In [19]:
y_pred = (model.predict(x_test) > 0.5).astype("int32")

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Accuracy: 0.85625
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.36      0.51       134
           1       0.85      0.99      0.92       506

    accuracy                           0.86       640
   macro avg       0.87      0.67      0.71       640
weighted avg       0.86      0.86      0.83       640



In [20]:
cm = confusion_matrix(y_test, y_pred)
fig = px.imshow(cm, text_auto=True, title="Confusion Matrix")
fig.show()

Save the model

In [22]:
model.save('/kaggle/working/cnn_model.h5')